### Let"s create a visualisation for the groupschat of a field hockey team :)

Import the used packages.

In [ ]:
from pathlib import Path
from loguru import logger
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tomllib
import numpy as np
import openpyxl
import re


Use the same way as the notebook in the course folder to get the data.

In [ ]:
configfile = Path("../config.toml").resolve()
with configfile.open("rb") as f:
    config = tomllib.load(f)
config

In [ ]:
root = Path("..").resolve()
processed = root / Path(config["processed"])
raw = root / Path(config["raw"])
datafile = processed / config["current"]
role_file = raw / config["role_file"]


In [ ]:
merged_df = pd.read_parquet(datafile)

emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE,
)

def count_emojis(text):
    return len(emoji_pattern.findall(text))

merged_df["emoji_count"] = merged_df["message"].apply(count_emojis)


merged_df["message_length"] = merged_df["message"].str.len()
merged_df['prev_author'] = merged_df['author'].shift(1)
merged_df['prev_timestamp'] = merged_df['timestamp'].shift(1)
merged_df['time_since_prev_sec'] = (merged_df['timestamp'] - merged_df['prev_timestamp']).dt.total_seconds()
merged_df['time_since_prev_min'] = (merged_df['timestamp'] - merged_df['prev_timestamp']).dt.total_seconds() / 60
merged_df.head()
print(merged_df.columns)

In [ ]:
# author_matrix = merged_df[merged_df['author'] != merged_df['prev_author']].copy()
# author_matrix = merged_df.pivot_table(index='author', columns='prev_author', values='message', aggfunc='count', fill_value=0)

# plt.figure(figsize=(10, 8))
# sns.heatmap(author_matrix, annot=True, fmt="d", cmap="YlGnBu")
# plt.title("Number of Messages from One Author to Previous Author")
# plt.xlabel("Previous Author")
# plt.ylabel("Author")
# plt.show()

In [ ]:
player_roles = pd.read_json(role_file, encoding = "latin")
player_roles.head()

In [ ]:
merged_df = pd.merge(merged_df, player_roles, left_on='author', right_on='Author')
merged_df = merged_df.drop(columns=['Author'])
merged_df['prev_position'] = merged_df['Position'].shift(1)
print(merged_df.columns)

In [ ]:
# Filter rijen waar er geen berichten naar elkaar worden gestuurd
filtered_df = merged_df[merged_df['prev_author'].notna()]
filtered_df = merged_df[merged_df['author'] != merged_df['prev_author']].copy()

# Maak een pivot table om het aantal berichten van de ene auteur naar de vorige auteur te tellen
author_matrix = filtered_df.pivot_table(index='Position', columns='prev_position', values='message', aggfunc='count', fill_value=0)

# Plot de matrix met een heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(author_matrix, annot=True, fmt="d", cmap="YlGnBu")
plt.title("Number of Messages from One Author to Previous Author")
plt.xlabel("Previous Author")
plt.ylabel("Author")
plt.show()

print(author_matrix.columns)

In [ ]:
# Zorg ervoor dat merged_df correct is ingeladen en verwerkt
# Filter rijen waar er geen berichten naar elkaar worden gestuurd
filtered_df = merged_df[merged_df['prev_author'].notna()]
filtered_df = merged_df[merged_df['author'] != merged_df['prev_author']].copy()

# Maak een pivot table om het aantal berichten van de ene auteur naar de vorige auteur te tellen
author_matrix = filtered_df.pivot_table(index='Position', columns='prev_position', values='message', aggfunc='count', fill_value=0)

# Bewaar de originele matrix met aantallen
author_matrix_counts = author_matrix.copy()

# Bereken percentages per rij (elke rij telt op tot 100%)
author_matrix_percentages = author_matrix.div(author_matrix.sum(axis=1), axis=0)
author_matrix_percentages = author_matrix_percentages.round(3)  # Rond af op 1 decimaal

# Define the desired order for sorting
desired_order = ['Keeper', 'Verdediger', 'Middenvelder', 'Aanvaller', 'Staff']

# Reindex the matrix to match the desired order
author_matrix_percentages = author_matrix_percentages.reindex(index=desired_order, columns=desired_order)

# Groeperen op 'Position' en tellen van unieke 'author' en het aantal berichten, en combineer de resultaten in een enkele DataFrame
overzicht_df = pd.DataFrame({
    'Aantal authors': merged_df.groupby('Position')['author'].nunique(),
    'Aantal berichten': merged_df.groupby('Position')['message'].count()
})

# Reindex het DataFrame op basis van de gewenste volgorde
overzicht_df = overzicht_df.reindex(desired_order)

# Maak de zin gebaseerd op het overzicht DataFrame
total_messages = overzicht_df['Aantal berichten'].sum()
sentence_parts = [f"Gebaseerd op {total_messages:,} berichten, waarbij iemand niet op zichzelf heeft gereageerd.\n"]

for position, row in overzicht_df.iterrows():
    sentence_parts.append(f"{position}: {row['Aantal berichten']:,} berichten verzonden door {row['Aantal authors']} auteurs\n")

sentence = "".join(sentence_parts)

# Vervang komma's door punten
sentence = sentence.replace(",", ".")

# Plot de percentages met een heatmap, waarbij de x-as en y-as worden omgedraaid, de streepjes bij de assen worden verwijderd, een grid wordt toegevoegd en de vakjes kleiner worden gemaakt
plt.figure(figsize=(10, 6))
sns.heatmap(author_matrix_percentages, annot=True, fmt=".3f", cmap="PuBu", linewidths=.5, cbar=False)
plt.suptitle('Ook in de groepschat zetten de verdedigers de lijnen uit', fontsize=16, fontweight= 'bold')
plt.title("Grootste kans op een reply van een verdediger")
plt.xlabel("Beantwoorder", labelpad=20, fontweight="bold", fontsize = 12)  # Voeg padding toe aan x-as label
plt.ylabel("Verzender", labelpad=20, fontweight="bold", fontsize = 12)  # Voeg padding toe aan y-as label
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.gca().xaxis.set_ticks_position('none') 
plt.gca().yaxis.set_ticks_position('none')



# Voeg figtext toe met het overzicht van het aantal auteurs en berichten per positie
plt.figtext(0.0, -0.2, sentence, wrap=True, horizontalalignment='left', fontsize=10)
# Voeg witruimte toe aan de rechterkant van de visual
plt.subplots_adjust(right=0.75)
plt.savefig("Relationships.png", bbox_inches='tight')
plt.show()




In [ ]:
# Groeperen op 'Position' en tellen van unieke 'author' en het aantal berichten, en combineer de resultaten in een enkele DataFrame
overzicht_df = pd.DataFrame({
    'Aantal authors': merged_df.groupby('Position')['author'].nunique(),
    'Aantal berichten': merged_df.groupby('Position')['message'].count()
})

# Definieer de gewenste volgorde
desired_order = ['Keeper', 'Verdediger', 'Middenvelder', 'Aanvaller', 'Staff']

# Reindex het DataFrame op basis van de gewenste volgorde
overzicht_df = overzicht_df.reindex(desired_order)

# Maak de zin gebaseerd op het overzicht DataFrame
total_messages = overzicht_df['Aantal berichten'].sum()
sentence_parts = [f"Gebaseerd op {total_messages:,} berichten\n"]

for position, row in overzicht_df.iterrows():
    sentence_parts.append(f"{position}: {row['Aantal berichten']:,} berichten verzonden door {row['Aantal authors']} auteurs\n")

sentence = "".join(sentence_parts)

# Vervang komma's door punten
sentence = sentence.replace(",", ".")

print(sentence)

In [ ]:
# Groeperen op 'Position' en tellen van unieke 'author' en het aantal berichten, en combineer de resultaten in een enkele DataFrame
overzicht_df = pd.DataFrame({
    'Aantal authors': merged_df.groupby('Position')['author'].nunique(),
    'Aantal berichten': merged_df.groupby('Position')['message'].count()
})

# Maak de zin gebaseerd op het overzicht DataFrame
total_messages = overzicht_df['Aantal berichten'].sum()
sentence_parts = [f"Gebaseerd op {total_messages:,} berichten:\n"]

for position, row in overzicht_df.iterrows():
    sentence_parts.append(f"waarvan {row['Aantal berichten']:,} berichten met positie {position} verzonden door {row['Aantal authors']} auteurs\n")

sentence = "".join(sentence_parts)
# Vervang komma's door punten
sentence = sentence.replace(",", ".")

print(sentence)

In [ ]:
# Zorg ervoor dat merged_df correct is ingeladen en verwerkt
# Filter rijen waar er geen berichten naar elkaar worden gestuurd
filtered_df = merged_df[merged_df['author'] != merged_df['prev_author']].copy()
# filtered_df = filtered_df[filtered_df['prev_author'].notna()]

# Maak een pivot table om het aantal berichten van de ene auteur naar de vorige auteur te tellen
author_matrix = filtered_df.pivot_table(index='Position', columns='prev_position', values='message', aggfunc='count', fill_value=0)

# Bereken de percentages van de berichten in de matrix zodat de som van de percentages per categorie 100% is
author_matrix_percentage = author_matrix.div(author_matrix.sum(axis=1), axis=0) * 100

# Plot de matrix met een heatmap en plaats de x-as waarden en titel bovenaan
plt.figure(figsize=(12, 10))
sns.heatmap(author_matrix_percentage, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'orientation': 'horizontal'})
plt.title("Percentage of Messages from One Author to Previous Author")
plt.xlabel("Author")
plt.ylabel("Replier")
plt.xticks(rotation=90)
plt.gca().xaxis.tick_top()  # Plaats de x-as waarden bovenaan
plt.gca().xaxis.set_label_position('top')  # Plaats de x-as titel bovenaan

# Verwijder de streepjes van de assen
plt.tick_params(axis='both', which='both', length=0)

plt.show()

In [ ]:
# df = merged_df[merged_df['author'] != merged_df['prev_author']].copy()
df = merged_df

p = (
    df.groupby(["author", "Position"])
    .agg({"message_length": "mean", "time_since_prev_min": "mean", "has_emoji": "mean", "author": "count"})
    .rename(columns={"author": "count"})
)

p = p[p["count"] > 10]
sns.scatterplot(data=p, x="message_length", y="has_emoji", hue = "Position", alpha=0.5)
print(p.columns)

In [ ]:
scatter_plot = sns.scatterplot(data=p, x="message_length", y="has_emoji", size="count", sizes=(10, 500), alpha=0.3, hue="Position", palette="bright")

# Verplaats de legenda buiten de grafiek
scatter_plot.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Position")
print(p.columns)

In [ ]:
# Reset de index zodat author en Position gewone kolommen worden
p = p.reset_index()

# Nu kun je gewoon je scatterplot maken
scatter_plot = sns.scatterplot(
    data=p, 
    x="message_length", 
    y="has_emoji", 
    size="count", 
    sizes=(10, 500), 
    alpha=0.5,
    hue="Position",
    palette="Dark2"  # Duidelijkere kleuren
)

# Verwijder de standaard legenda
scatter_plot.legend_.remove()

# Maak een nieuwe legenda die alleen de Position-variabele toont
handles, labels = scatter_plot.get_legend_handles_labels()
num_positions = len(p["Position"].unique())
position_handles = handles[-num_positions:]
position_labels = labels[-num_positions:]

# Voeg de nieuwe legenda toe
scatter_plot.legend(
    position_handles, 
    position_labels, 
    loc='center left', 
    bbox_to_anchor=(1, 0.5), 
    title="Position"
)